In [1]:
# 1. 데이터 다운로드 (Colab 환경)
!wget -O data.tar.gz "https://aistages-api-public-prod.s3.amazonaws.com/app/Competitions/000377/data/data.tar.gz"
!tar -xzf data.tar.gz
!pip install -q segmentation-models-pytorch albumentations opencv-python-headless

--2026-02-06 10:47:55--  https://aistages-api-public-prod.s3.amazonaws.com/app/Competitions/000377/data/data.tar.gz
Resolving aistages-api-public-prod.s3.amazonaws.com (aistages-api-public-prod.s3.amazonaws.com)... 3.5.185.69, 52.219.146.51, 3.5.187.10, ...
Connecting to aistages-api-public-prod.s3.amazonaws.com (aistages-api-public-prod.s3.amazonaws.com)|3.5.185.69|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3280795723 (3.1G) [binary/octet-stream]
Saving to: ‘data.tar.gz’

data.tar.gz         100%[===================>]   3.05G  47.3MB/s    in 71s     

2026-02-06 10:49:06 (44.3 MB/s) - ‘data.tar.gz’ saved [3280795723/3280795723]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 154.8/154.8 kB 13.0 MB/s eta 0:00:00


In [4]:
!pip install pyclipper shapely

  Using cached pyclipper-1.4.0-cp312-cp312-manylinux2014_x86_64.manylinux_2_17_x86_64.whl.metadata (8.6 kB)
Using cached pyclipper-1.4.0-cp312-cp312-manylinux2014_x86_64.manylinux_2_17_x86_64.whl (978 kB)


In [5]:
import os
import json
import cv2
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader, ConcatDataset
from torch.amp import autocast, GradScaler
import albumentations as A
from albumentations.pytorch import ToTensorV2
import segmentation_models_pytorch as smp
import pyclipper
from shapely.geometry import Polygon
from tqdm import tqdm
import ssl
import warnings

# --- 환경 설정 ---
warnings.filterwarnings('ignore')
ssl._create_default_https_context = ssl._create_unverified_context
os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "expandable_segments:True"
DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# --- [하이퍼파라미터] ---
# 1. 모델 설정
ENCODER_NAME = "tu-convnext_base" # 성능 최강 백본
MODEL_NAME = "DBNet_ConvNeXt_FPN"

# 2. 학습 설정
IMG_SIZE = 1024
BATCH_SIZE = 8
EPOCHS = 25
LR = 1e-3

# 3. DBNet 핵심 파라미터 (신의 한 수)
SHRINK_RATIO = 0.4  # 학습 시: 글자 영역을 40% 쪼그라트림 (분리 유도)
UNCLIP_RATIO = 1.8  # 추론 시: 쪼그라든걸 1.8배 다시 키움 (영역 확보)
BOX_THRESH = 0.5    # 확신도 0.5 이상만 인정

# --- [경로 설정] ---
BASE_PATH = './data/datasets'
PSEUDO_PATH = './data/pseudo_label'
OUTPUT_CSV = 'submission_dbnet_final.csv'

# ==========================================
# 1. Dataset 클래스 (학습용 & 추론용)
# ==========================================
class DBDataset(Dataset):
    def __init__(self, img_dir, json_path, transform=None, mode='train'):
        self.img_dir = img_dir
        self.transform = transform
        self.mode = mode

        # JSON 로드 (없으면 예외처리)
        if json_path and not os.path.exists(json_path):
            alt = json_path.replace('train.json', 'val.json')
            if os.path.exists(alt): json_path = alt
            else:
                self.image_names = []
                return

        with open(json_path, 'r', encoding='utf-8') as f:
            self.data = json.load(f)['images']
        self.image_names = list(self.data.keys())

    def __len__(self): return len(self.image_names)

    def __getitem__(self, idx):
        name = self.image_names[idx]
        image = cv2.imread(os.path.join(self.img_dir, name))
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

        # 추론 모드일 경우 원본 크기 저장 필요
        orig_h, orig_w = image.shape[:2]

        # 마스크 생성 (학습용)
        mask = np.zeros((orig_h, orig_w), dtype=np.float32)

        if self.mode == 'train' and 'words' in self.data[name]:
            for w_info in self.data[name]['words'].values():
                pts = np.array(w_info['points'], dtype=np.int32)

                # [DBNet 학습의 핵심: Shrink]
                # 글자를 안쪽으로 축소시켜서 마스크를 만듦 -> 인접 글자 분리 효과
                try:
                    poly = Polygon(pts)
                    area = poly.area
                    perimeter = poly.length

                    if area > 0 and perimeter > 0:
                        distance = area * (1 - SHRINK_RATIO * SHRINK_RATIO) / perimeter
                        offset = pyclipper.PyclipperOffset()
                        offset.AddPath(pts, pyclipper.JT_ROUND, pyclipper.ET_CLOSEDPOLYGON)
                        shrunk = offset.Execute(-distance) # 마이너스 거리 = 축소

                        if len(shrunk) > 0:
                            cv2.fillPoly(mask, [np.array(shrunk[0], dtype=np.int32)], 1)
                        else:
                            cv2.fillPoly(mask, [pts], 1) # 너무 작아지면 원본 사용
                except:
                    cv2.fillPoly(mask, [pts], 1)

        if self.transform:
            aug = self.transform(image=image, mask=mask)
            image = aug['image']
            mask = aug['mask']

        if self.mode == 'test':
            return image, name, (orig_h, orig_w)
        else:
            return image, mask

# ==========================================
# 2. Augmentation
# ==========================================
train_transform = A.Compose([
    A.Resize(IMG_SIZE, IMG_SIZE),
    A.Perspective(scale=(0.05, 0.1), p=0.5),
    A.Rotate(limit=20, p=0.5),
    A.ColorJitter(brightness=0.2, contrast=0.2, p=0.4),
    A.Normalize(),
    ToTensorV2()
])

test_transform = A.Compose([
    A.Resize(IMG_SIZE, IMG_SIZE),
    A.Normalize(),
    ToTensorV2()
])

# ==========================================
# 3. 학습 함수
# ==========================================
def train():
    print(f"🔥 DBNet 학습 시작! (Encoder: {ENCODER_NAME})")
    print(f"   - Shrink Ratio: {SHRINK_RATIO} (글자를 쪼그라트려 학습)")

    # 데이터 로드
    train_main = DBDataset(os.path.join(BASE_PATH, 'images/train'), os.path.join(BASE_PATH, 'jsons/train.json'), transform=train_transform, mode='train')
    pseudo_datasets = []
    for folder in ['sroie', 'cord-v2', 'wildreceipt']:
        p_img = os.path.join(PSEUDO_PATH, folder, 'images')
        p_json = os.path.join(PSEUDO_PATH, folder, 'train.json')
        if os.path.exists(p_json):
            pseudo_datasets.append(DBDataset(p_img, p_json, transform=train_transform, mode='train'))

    full_ds = ConcatDataset([train_main] + pseudo_datasets)
    train_loader = DataLoader(full_ds, batch_size=BATCH_SIZE, shuffle=True, num_workers=4, pin_memory=True, drop_last=True)

    # 모델 정의: FPN 사용 (DBNet 필수)
    model = smp.FPN(
        encoder_name=ENCODER_NAME,
        encoder_weights="imagenet",
        in_channels=3,
        classes=1,
    ).to(DEVICE)

    optimizer = torch.optim.AdamW(model.parameters(), lr=LR, weight_decay=1e-2)
    scheduler = torch.optim.lr_scheduler.OneCycleLR(optimizer, max_lr=LR, epochs=EPOCHS, steps_per_epoch=len(train_loader))

    # Loss: 경계 분리를 위해 Dice Loss 비중 높임
    dice_loss = smp.losses.DiceLoss(mode='binary')
    bce_loss = smp.losses.SoftBCEWithLogitsLoss()
    scaler = GradScaler('cuda')

    for epoch in range(1, EPOCHS + 1):
        model.train()
        epoch_loss = 0
        pbar = tqdm(train_loader, desc=f"Epoch {epoch}/{EPOCHS}")

        for imgs, msks in pbar:
            imgs, msks = imgs.to(DEVICE), msks.to(DEVICE).unsqueeze(1)
            optimizer.zero_grad()

            with autocast('cuda', dtype=torch.bfloat16):
                preds = model(imgs)
                loss = dice_loss(preds, msks) + bce_loss(preds, msks)

            scaler.scale(loss).backward()
            scaler.step(optimizer)
            scaler.update()
            scheduler.step()
            epoch_loss += loss.item()
            pbar.set_postfix(loss=loss.item())

    # 모델 저장
    save_path = f"{MODEL_NAME}_final.pth"
    torch.save(model.state_dict(), save_path)
    print(f"✅ 학습 완료! 모델 저장됨: {save_path}")
    return save_path

# ==========================================
# 4. 추론 및 후처리 함수 (Unclip)
# ==========================================
def unclip(box, unclip_ratio):
    """쪼그라든 영역을 다시 팽창시키는 DBNet 핵심 로직"""
    poly = Polygon(box)
    area = poly.area
    length = poly.length
    if area <= 0 or length <= 0: return box

    # 팽창시킬 거리 계산
    distance = area * unclip_ratio / length

    offset = pyclipper.PyclipperOffset()
    offset.AddPath(box, pyclipper.JT_ROUND, pyclipper.ET_CLOSEDPOLYGON)
    expanded = offset.Execute(distance) # 플러스 거리 = 팽창

    return expanded

def inference(model_path):
    print(f"🚀 추론 및 파일 생성 시작... (Unclip Ratio: {UNCLIP_RATIO})")

    # 모델 로드
    model = smp.FPN(encoder_name=ENCODER_NAME, in_channels=3, classes=1).to(DEVICE)
    model.load_state_dict(torch.load(model_path, map_location=DEVICE))
    model.eval()

    # 데이터 로드
    test_ds = DBDataset(os.path.join(BASE_PATH, 'images/test'), os.path.join(BASE_PATH, 'jsons/test.json'), transform=test_transform, mode='test')
    test_loader = DataLoader(test_ds, batch_size=BATCH_SIZE, shuffle=False, num_workers=4)

    results = {}

    with torch.no_grad():
        for imgs, names, (orig_hs, orig_ws) in tqdm(test_loader, desc="Inference"):
            imgs = imgs.to(DEVICE)

            # TTA: 좌우 반전 앙상블
            with autocast('cuda', dtype=torch.bfloat16):
                p1 = torch.sigmoid(model(imgs))
                p2 = torch.sigmoid(model(torch.flip(imgs, dims=[3])))
                preds = (p1 + torch.flip(p2, dims=[3])) / 2

            preds = preds.float().cpu().numpy()

            for i, name in enumerate(names):
                orig_h = orig_hs[i].item()
                orig_w = orig_ws[i].item()
                pred_mask = preds[i][0]

                # 1. 원본 크기 복원
                pred_mask = cv2.resize(pred_mask, (orig_w, orig_h))

                # 2. 이진화
                binary_mask = (pred_mask > BOX_THRESH).astype(np.uint8)

                # 3. 윤곽선 검출
                contours, _ = cv2.findContours(binary_mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

                polygons = []
                for cnt in contours:
                    if cv2.contourArea(cnt) < 50: continue # 노이즈 제거

                    epsilon = 0.005 * cv2.arcLength(cnt, True)
                    approx = cv2.approxPolyDP(cnt, epsilon, True)

                    if len(approx) < 3: continue

                    reshaped_poly = approx.reshape(-1, 2)

                    # 4. [DBNet Post-processing] Unclip (팽창)
                    # 학습 때 줄어든 만큼, 혹은 그 이상으로 키워서 Recall 확보 + Precision 유지
                    try:
                        expanded = unclip(reshaped_poly, UNCLIP_RATIO)
                        if len(expanded) > 0:
                            final_poly = np.array(expanded[0])
                            polygons.append(final_poly.reshape(-1).tolist())
                        else:
                            polygons.append(reshaped_poly.reshape(-1).tolist())
                    except:
                        polygons.append(reshaped_poly.reshape(-1).tolist())

                results[name] = "|".join([" ".join(map(str, poly)) for poly in polygons])

    # 파일 생성
    sample = pd.read_csv(os.path.join(BASE_PATH, 'sample_submission.csv'))
    sample['polygons'] = sample['filename'].map(results).fillna("")
    sample.to_csv(OUTPUT_CSV, index=False)
    print(f"🎉 모든 작업 완료! 제출 파일: {OUTPUT_CSV}")

if __name__ == "__main__":
    # 1. 학습 실행
    saved_model = train()

    # 2. 바로 추론 실행
    inference(saved_model)

🔥 DBNet 학습 시작! (Encoder: tu-convnext_base)
   - Shrink Ratio: 0.4 (글자를 쪼그라트려 학습)


model.safetensors:   0%|          | 0.00/354M [00:00<?, ?B/s]

Epoch 25/25: 100%|██████████| 409/409 [02:08<00:00,  3.19it/s, loss=0.223]


✅ 학습 완료! 모델 저장됨: DBNet_ConvNeXt_FPN_final.pth
🚀 추론 및 파일 생성 시작... (Unclip Ratio: 1.8)


Inference: 100%|██████████| 52/52 [00:21<00:00,  2.44it/s]


🎉 모든 작업 완료! 제출 파일: submission_dbnet_final.csv


In [6]:
import os
import json
import cv2
import numpy as np
import pandas as pd
import torch
from torch.utils.data import Dataset, DataLoader
from torch.amp import autocast
import albumentations as A
from albumentations.pytorch import ToTensorV2
import segmentation_models_pytorch as smp
import pyclipper
from shapely.geometry import Polygon
from tqdm import tqdm
import ssl
import warnings

warnings.filterwarnings('ignore')
ssl._create_default_https_context = ssl._create_unverified_context

# --- [튜닝 핵심] Recall 극대화 설정 ---
ENCODER_NAME = "tu-convnext_base"
# 기존에 학습 완료한 모델 경로 (파일명 확인 필요!)
MODEL_PATH = "DBNet_ConvNeXt_FPN_final.pth"
OUTPUT_CSV = "submission_dbnet_high_recall.csv"

# 🚨 [여기를 주목하세요] 🚨
# Precision 0.98을 믿고 아주 과감하게 세팅합니다.
UNCLIP_RATIO = 3.0  # (기존 1.8 -> 3.0) 쪼그라든 영역을 3배로 확 불립니다!
BOX_THRESH = 0.3    # (기존 0.5 -> 0.3) 희미한 글자도 30% 확신만 있으면 잡습니다.

BASE_PATH = './data/datasets'
TEST_IMG_DIR = os.path.join(BASE_PATH, 'images/test')
TEST_JSON = os.path.join(BASE_PATH, 'jsons/test.json')
SAMPLE_SUB = os.path.join(BASE_PATH, 'sample_submission.csv')

DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
IMG_SIZE = 1024

class TestDataset(Dataset):
    def __init__(self, img_dir, json_path, transform=None):
        self.img_dir = img_dir
        self.transform = transform
        with open(json_path, 'r', encoding='utf-8') as f:
            self.data = json.load(f)['images']
        self.image_names = list(self.data.keys())

    def __len__(self): return len(self.image_names)

    def __getitem__(self, idx):
        name = self.image_names[idx]
        image = cv2.imread(os.path.join(self.img_dir, name))
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        orig_h, orig_w = image.shape[:2]
        if self.transform:
            image = self.transform(image=image)['image']
        return image, name, (orig_h, orig_w)

def unclip(box, unclip_ratio):
    poly = Polygon(box)
    area = poly.area
    length = poly.length
    if area <= 0 or length <= 0: return box

    # 거리 계산: 면적 대비 둘레 비율로 확장 거리 결정
    distance = area * unclip_ratio / length

    offset = pyclipper.PyclipperOffset()
    offset.AddPath(box, pyclipper.JT_ROUND, pyclipper.ET_CLOSEDPOLYGON)
    expanded = offset.Execute(distance)

    return expanded

def preds_to_polygons(pred_mask, orig_h, orig_w):
    # 1. 원본 크기 복원
    pred_mask = cv2.resize(pred_mask, (orig_w, orig_h))

    # 2. 이진화 (낮아진 Threshold 적용)
    binary_mask = (pred_mask > BOX_THRESH).astype(np.uint8)

    # 3. 윤곽선 검출
    contours, _ = cv2.findContours(binary_mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    polygons = []
    for cnt in contours:
        if cv2.contourArea(cnt) < 30: continue # 아주 작은 노이즈만 제거

        epsilon = 0.003 * cv2.arcLength(cnt, True) # 더 정교하게 (0.005 -> 0.003)
        approx = cv2.approxPolyDP(cnt, epsilon, True)

        if len(approx) < 3: continue

        reshaped_poly = approx.reshape(-1, 2)

        # 4. Unclip (확장) 적용
        try:
            expanded = unclip(reshaped_poly, UNCLIP_RATIO)
            if len(expanded) > 0:
                final_poly = np.array(expanded[0])
                polygons.append(final_poly.reshape(-1).tolist())
            else:
                polygons.append(reshaped_poly.reshape(-1).tolist())
        except:
            polygons.append(reshaped_poly.reshape(-1).tolist())

    return "|".join([" ".join(map(str, poly)) for poly in polygons])

def main():
    print(f"🔥 Recall 부스팅 시작! (Unclip: {UNCLIP_RATIO}, Thresh: {BOX_THRESH})")

    model = smp.FPN(
        encoder_name=ENCODER_NAME,
        encoder_weights=None,
        in_channels=3,
        classes=1,
    ).to(DEVICE)

    if not os.path.exists(MODEL_PATH):
        print(f"❌ 모델 파일({MODEL_PATH})이 없습니다.")
        return

    model.load_state_dict(torch.load(MODEL_PATH, map_location=DEVICE))
    model.eval()

    test_transform = A.Compose([
        A.Resize(IMG_SIZE, IMG_SIZE),
        A.Normalize(),
        ToTensorV2()
    ])

    test_ds = TestDataset(TEST_IMG_DIR, TEST_JSON, transform=test_transform)
    test_loader = DataLoader(test_ds, batch_size=8, shuffle=False, num_workers=4)

    results = {}

    with torch.no_grad():
        for imgs, names, (orig_hs, orig_ws) in tqdm(test_loader):
            imgs = imgs.to(DEVICE)

            with autocast('cuda', dtype=torch.bfloat16):
                p1 = torch.sigmoid(model(imgs))
                p2 = torch.sigmoid(model(torch.flip(imgs, dims=[3])))
                preds = (p1 + torch.flip(p2, dims=[3])) / 2

            preds = preds.float().cpu().numpy()

            for i, name in enumerate(names):
                orig_h = orig_hs[i].item()
                orig_w = orig_ws[i].item()

                poly_str = preds_to_polygons(preds[i][0], orig_h, orig_w)
                results[name] = poly_str

    sample_df = pd.read_csv(SAMPLE_SUB)
    sample_df['polygons'] = sample_df['filename'].map(results).fillna("")
    sample_df.to_csv(OUTPUT_CSV, index=False)
    print(f"🎉 파일 생성 완료: {OUTPUT_CSV}")
    print("👉 이 파일은 '너무 높았던 Precision'을 조금 희생하고 Recall을 대폭 가져옵니다.")

if __name__ == "__main__":
    main()

🔥 Recall 부스팅 시작! (Unclip: 3.0, Thresh: 0.3)


100%|██████████| 52/52 [00:19<00:00,  2.65it/s]


🎉 파일 생성 완료: submission_dbnet_high_recall.csv
👉 이 파일은 '너무 높았던 Precision'을 조금 희생하고 Recall을 대폭 가져옵니다.
